In [1]:
from scripts.form_matrix import Hexagonal_Prism


In [2]:
# Defining parameters

a = 1
h = 0.5
rho = 1

prism = Hexagonal_Prism(a, rho, h)

uniform_IT = prism.form_uniform_matrix()[0]
composite_IT = prism.form_matrix_triangles()[0]

print(uniform_IT)
print(composite_IT)

[[ 2.97696233e-01 -5.20417043e-17  5.63785130e-18]
 [-5.20417043e-17  2.97696233e-01  3.03576608e-18]
 [ 5.63785130e-18  3.03576608e-18  5.41265877e-01]]
[[ 2.97696233e-01 -5.55111512e-17 -9.87234988e-19]
 [-5.55111512e-17  2.97696233e-01  9.92956029e-19]
 [-9.87234988e-19  9.92956029e-19  5.41265877e-01]]
